In [1]:
import pandas as pd

from tradier_funcs import *

# Set the API key and base URL
ACCOUNT_ID = '6YB53834'
API_KEY = 'xfiiBZaKv6Z68ossl3VjtjWL0Tsc'
BASE_URL = 'https://api.tradier.com/'

In [2]:
date_list = [
    "2025-10-31",
    "2025-11-03",
    "2025-11-04",
    "2025-11-05",
    "2025-11-06",
    "2025-11-07"
]
index_list = ["greeks.updated_at", "expiration_date", "option_type", "strike", ]

df = pd.DataFrame()
for date in date_list:
    chains = get_chains("SPX", date)
    chains["gex"] = (
        chains["strike"] * chains["greeks.gamma"] * chains["open_interest"] * 100
    )

    chains.loc[chains["option_type"] == "put", "gex"] *= -1
    chains.set_index(index_list, inplace=True)
    df = pd.concat([df, chains])

Index(['symbol', 'description', 'exch', 'type', 'last', 'change', 'volume',
       'open', 'high', 'low', 'close', 'bid', 'ask', 'underlying', 'strike',
       'change_percentage', 'average_volume', 'last_volume', 'trade_date',
       'prevclose', 'week_52_high', 'week_52_low', 'bidsize', 'bidexch',
       'bid_date', 'asksize', 'askexch', 'ask_date', 'open_interest',
       'contract_size', 'expiration_date', 'expiration_type', 'option_type',
       'root_symbol', 'greeks.delta', 'greeks.gamma', 'greeks.theta',
       'greeks.vega', 'greeks.rho', 'greeks.phi', 'greeks.bid_iv',
       'greeks.mid_iv', 'greeks.ask_iv', 'greeks.smv_vol',
       'greeks.updated_at'],
      dtype='object')
Index(['symbol', 'description', 'exch', 'type', 'last', 'change', 'volume',
       'open', 'high', 'low', 'close', 'bid', 'ask', 'underlying', 'strike',
       'change_percentage', 'average_volume', 'last_volume', 'trade_date',
       'prevclose', 'week_52_high', 'week_52_low', 'bidsize', 'bidexch',
    

In [3]:
import sqlite3

# Create a connection to the SQLite database
conn = sqlite3.connect('gex_data.db')

# Query the most recent greeks.updated_at timestamp from the SQLite table
query = "SELECT MAX([greeks.updated_at]) AS max_updated_at FROM gex_table"
most_recent_timestamp = pd.read_sql(query, conn)['max_updated_at'].iloc[0]
# print(most_recent_timestamp)

# Get the most recent greeks.updated_at timestamp from the API query dataframe
api_most_recent_timestamp = chains.index.get_level_values('greeks.updated_at').max()
# print(api_most_recent_timestamp)

# Save the dataframe to a SQL table if the timestamps are different
if pd.to_datetime(most_recent_timestamp) != pd.to_datetime(api_most_recent_timestamp):
    df.to_sql('gex_table', conn, if_exists='append', index=True)
    print("Data added.")
else:
    print("No change.")

No change.


In [4]:
# Read the SQL table back into a dataframe
df = pd.read_sql('SELECT * FROM gex_table', conn)
conn.close()

# Write to CSV for dashboard
df.to_csv('gex.csv', index=False)

In [5]:
prices = get_historical_quotes(['SPX'], start_date='2025-03-01', end_date='2025-12-31', resolution='daily')
prices.drop(['symbol', 'volume'], inplace=True, axis=1)
prices.to_csv('spx_prices.csv', index=False)

In [6]:
def nearest_multiples_of_5(value):
    lower_multiple = (value // 5) * 5
    higher_multiple = lower_multiple + 5
    return lower_multiple, higher_multiple

multiples = prices["close"].apply(nearest_multiples_of_5)


In [7]:
get_latest_quotes(['SPX']).T

,0
symbol,SPX
description,S&P 500 Index (Derived)
exch,C
type,index
last,6860.82
change,38.48
volume,0
open,None
high,None
low,None
